# San Francisco Housing Rental Analysis

In this assignment, you will perform basic analysis for the San Francisco Housing Market to allow potential real estate investors to choose rental investment properties. 

In [ ]:
# initial imports
import os
import pandas as pd
import matplotlib.pyplot as plt
import plotly.express as px
import hvplot.pandas
from pathlib import Path
from dotenv import load_dotenv

%matplotlib inline

In [ ]:
# Read the Mapbox API key
load_dotenv()
mapbox_token = os.getenv("MAPBOX_API_KEY")
px.set_mapbox_access_token(mapbox_token)

## Load Data

In [ ]:
# Read the census data into a Pandas DataFrame
file_path = Path("Data/sfo_neighborhoods_census_data.csv")
sfo_data = pd.read_csv(file_path, index_col="year")
sfo_data.head()

- - - 

## Housing Units Per Year

In this section, you will calculate the number of housing units per year and visualize the results as a bar chart using the Pandas plot function. 

Hint: Use the Pandas groupby function

Optional challenge: Use the min, max, and std to scale the y limits of the chart.

In [ ]:
# Calculate the mean number of housing units per year (hint: use groupby) 
sfo_unit_ave = sfo_data['housing_units'].groupby(['year']).mean()
sfo_unit_ave

In [ ]:
# Use the Pandas plot function to plot the average housing units per year.
# Note: You will need to manually adjust the y limit of the chart using the min and max values from above.
fig_housing_units = sfo_unit_ave.hvplot.bar(
    xlabel='Year',
    ylabel='Housing Units',
    rot=45,
    ylim=(370000, 385000),
    yformatter='%0f',
    title='Housing Units in San Francisco from 2010 to 2016')

fig_housing_units

- - - 

## Average Prices per Square Foot

In this section, you will calculate the average gross rent and average sales price for each year. Plot the results as a line chart.

### Average Gross Rent in San Francisco Per Year

In [ ]:
# Calculate the average gross rent and average sale price per square foot
sfo_ave_spsf = sfo_data[{'sale_price_sqr_foot', 'gross_rent'}].groupby(['year']).mean()

# re-organizing the columns to match order in starter code:
sfo_ave_spsf.reindex(columns=['sale_price_sqr_foot', 'gross_rent'])
sfo_ave_spsf

In [ ]:
# Plot the Average Gross Rent per Year as a Line Chart 
ave_gross_rent = sfo_ave_spsf['gross_rent'].hvplot.line(
    xlabel='Year',
    ylabel='Gross Rent',
    rot=45,
    ylim=(1000, 4500),
    title='Average Gross Rent in San Francisco')

ave_gross_rent

### Average Sales Price per Year

In [ ]:
# Plot the Average Sales Price per Year as a line chart
ave_sales_price = sfo_ave_spsf['sale_price_sqr_foot'].hvplot.line(
    xlabel='Year',
    ylabel='Average Sale Price',
    rot=45,
    ylim=(300, 750),
    title='Average Sales Price Per Square Foot in San Francisco')

ave_sales_price

- - - 

## Average Prices by Neighborhood

In this section, you will use hvplot to create an interactive visulization of the Average Prices with a dropdown selector for the neighborhood.

Hint: It will be easier to create a new DataFrame from grouping the data and calculating the mean prices for each year and neighborhood

In [ ]:
# Group by year and neighborhood and then create a new dataframe of the mean values
sfo_neigh_mean = sfo_data[{'sale_price_sqr_foot', 'neighborhood'}].groupby(['year','neighborhood']).mean()

#sfo_neigh_mean.reset_index(inplace=True)
sfo_neigh_mean.reset_index(inplace=True)
sfo_neigh_mean.head()

In [ ]:
# Use hvplot to create an interactive line chart of the average price per sq ft.
sfo_neigh_mean.hvplot.line(
    x='year', 
    y='sale_price_sqr_foot', 
    width=600, 
    groupby='neighborhood', 
    widget_location='left_top',
    title='Average Price by Neighborhood')

- - - 

## The Top 10 Most Expensive Neighborhoods

In this section, you will need to calculate the mean sale price for each neighborhood and then sort the values to obtain the top 10 most expensive neighborhoods on average. Plot the results as a bar chart.

In [ ]:
# Getting the data from the top 10 expensive neighborhoods
# Creating dataframe with sale price per sq foot and neighborhood columns:
sfo_neigh_ave = sfo_data.groupby(['neighborhood']).mean()

# Sorting the dataframe in descending order to easily identify the top 10 most expensive:
sfo_neigh_ave.sort_values(by=['sale_price_sqr_foot'], ascending=False, inplace=True)

# Assigning the Top 10 most expensive on average to new dataframe/variable:
top_10 = sfo_neigh_ave.head(10)
top_10.reset_index()

In [ ]:
# Plotting the data from the top 10 expensive neighborhoods
top_10[{'sale_price_sqr_foot'}].hvplot.bar(rot=45, title='Top 10 Most Expensive Neighborhoods on Average')

- - - 

## Parallel Coordinates and Parallel Categories Analysis

In this section, you will use plotly express to create parallel coordinates and parallel categories visualizations so that investors can interactively filter and explore various factors related to the sales price of the neighborhoods. 

Using the DataFrame of Average values per neighborhood (calculated above), create the following visualizations:
1. Create a Parallel Coordinates Plot
2. Create a Parallel Categories Plot

In [ ]:
# resetting the index to prepare to incorporate into parallel coordinates & categories plots: 
top_10.reset_index(inplace=True)

# Parallel Coordinates Plot
px.parallel_coordinates(
    top_10,
    color= 'sale_price_sqr_foot')

In [ ]:
# Parallel Categories Plot:
px.parallel_categories(
    round(top_10,2),
    dimensions=['neighborhood','sale_price_sqr_foot', 'housing_units', 'gross_rent'],
    color= 'sale_price_sqr_foot',
    color_continuous_scale=px.colors.sequential.Inferno,
    labels={
        "neighborhood": "Neighborhood",
        "sale_price_sqr_foot": "Sale Price Per Sq Foot",
        "housing_units": "Housing Units",
        "gross_rent": "Gross Rent"        
    },
)

- - - 

## Neighborhood Map

In this section, you will read in neighboor location data and build an interactive map with the average prices per neighborhood. Use a scatter_mapbox from plotly express to create the visualization. Remember, you will need your mapbox api key for this.

### Load Location Data

In [ ]:
# Load neighborhoods coordinates data
file_path = Path("Data/neighborhoods_coordinates.csv")
df_neighborhood_locations = pd.read_csv(file_path)
df_neighborhood_locations.head()

### Data Preparation

You will need to join the location data with the mean prices per neighborhood

1. Calculate the mean values for each neighborhood
2. Join the average values with the neighborhood locations

In [ ]:
# Calculate the mean values for each neighborhood
# Already calculated mean in df sfo_neigh_ave so joined with location df on index 'neighborhood':
join_df = sfo_neigh_ave.join(df_neighborhood_locations.set_index('Neighborhood'))

# resetting the index so it can feed into the mapbox plot:
join_df.reset_index(inplace=True)
join_df.head()

##### Join the average values with the neighborhood locations
###### See my code in cell above

### Mapbox Visualization

Plot the aveage values per neighborhood with a plotly express scatter_mapbox visualization.

In [ ]:
# Create a scatter mapbox to analyze neighborhood info
map_plot = px.scatter_mapbox(
    round(join_df,2),
    title='Average Sale Price Per Square Foot and Gross Rent in San Francisco',
    lat="Lat",
    lon="Lon",
    size='sale_price_sqr_foot',
    color="gross_rent",
    hover_name='neighborhood',
    zoom=10
)

# Displaying the map:
map_plot.show()